In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score 
from sklearn.metrics import precision_recall_fscore_support, roc_curve, roc_auc_score

In [3]:
cancer_data = load_breast_cancer()
df = pd.DataFrame(cancer_data.data, columns = cancer_data['feature_names'])
df['target'] = cancer_data['target']
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
X = df[cancer_data.feature_names].values
y = df['target'].values

In [5]:
kf = KFold(n_splits = 5, shuffle = True)
for criterion in ['gini', 'entropy']:
    print('\tDecision Tree - {}'.format(criterion))
    accuracy = []
    precision = []
    recall = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = DecisionTreeClassifier(criterion = criterion)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy.append(model.score(X_test, y_test))
        precision.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))   
    print(np.mean(accuracy))
    print(np.mean(precision))
    print(np.mean(recall))

	Decision Tree - gini
0.9208818506443098
0.9410435625605904
0.9323630486512483
	Decision Tree - entropy
0.9367489520260829
0.9394973947812073
0.9601323444357501


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)
param_grid = { 
    'max_depth': [1, 3, 5 , 7, 15, 19, 25, 27, 35, 37, 50],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'max_leaf_nodes': [5, 7, 10, 17, 19, 15, 20, 25, 27, 30, 35, 39]
}
model = DecisionTreeClassifier()
gs = GridSearchCV(model, param_grid, scoring='f1', cv=5)
gs.fit(X_train, y_train)
print(gs.best_params_)

{'max_depth': 27, 'max_leaf_nodes': 27, 'min_samples_leaf': 1}


In [7]:
final_model = DecisionTreeClassifier(max_depth=gs.best_params_['max_depth'],
                                     min_samples_leaf=gs.best_params_['min_samples_leaf'],
                                     max_leaf_nodes=gs.best_params_['max_leaf_nodes'])
final_model.fit(X_train, y_train)
print("Accuracy: ", final_model.score(X_test, y_test))
y_pred = final_model.predict(X_test)
p, r, f, s = precision_recall_fscore_support(y_test, y_pred)
print("Precision: ", p[1])
print("Recall: ", r[1])
print("Fscore: ", f[1])

Accuracy:  0.9230769230769231
Precision:  0.9139784946236559
Recall:  0.9659090909090909
Fscore:  0.9392265193370166
